In [24]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
from methodsnm.solver import solve_on_freedofs
from methodsnm.forms import *
from methodsnm.formint import *
import numpy as np
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf
from methodsnm.fes import *
from numpy import exp

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.25))
T =3
mesh = UnstructuredHypertriangleMesh(T,ngmesh)
fes = P1_Hypertriangle_Space(mesh)


In [25]:
epsi = 0.001
wind =1
eps = GlobalFunction(lambda x: epsi, mesh = mesh)
w = ConstantVectorFunction(np.array([1,1,1,1]), mesh = mesh)

blf = BilinearForm(fes)
blf += LaplaceIntegral_without_time(eps)
blf += ConvectionIntegral(w)
blf.assemble()

f = lambda x: -(1 - exp((x[0]+x[1]+x[2] - 3)/epsi)) / (1 - exp(-1/epsi))* exp(-x[3])
f = GlobalFunction(f, mesh = mesh)
lf = LinearForm(fes)
lf += SourceIntegral(f)
lf.assemble()

top = mesh.top_bndry_vertices
initial = mesh.initial_bndry_vertices
bndry = list_diff(mesh.bndry_vertices, top)
freedofs = list_diff(mesh.vertices,bndry)

uex = lambda x: (1 - exp((x[0]+x[1]+x[2] - 3)/epsi)) / (1 - exp(-1/epsi)) * exp(-x[3])
uh = FEFunction(fes)
uh._set(uex, bndry)

res = lf.vector - blf.matrix.dot(uh.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2
u_exact = GlobalFunction(uex, mesh = mesh)
l2diff = compute_difference_L2(uh, u_exact, mesh, intorder = 5)
print("L2 difference:", l2diff)

L2 difference: 0.13633169044401267


In [26]:
blf += SUPGIntegral(w)
blf.assemble()

lf += SUPGSourceIntegral(f, w)
lf.assemble()

uh = FEFunction(fes)
uh._set(uex, bndry)

res = lf.vector - blf.matrix.dot(uh.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2
u_exact = GlobalFunction(uex, mesh = mesh)
l2diff = compute_difference_L2(uh, u_exact, mesh, intorder = 5)
print("L2 difference:", l2diff)

L2 difference: 0.02614878100206763
